In [25]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from itertools import combinations
from collections import defaultdict
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score

kic = pd.read_csv('kic.csv')
fac = pd.read_csv('전국지식산업센터_입주업체.csv')

fac.head(2)

/var/folders/gh/rmj80qln5tz2v55hd102xqhr0000gn/T/ipykernel_60417/1888101303.py:15: DtypeWarning: Columns (7,27) have mixed types. Specify dtype option on import or set low_memory=False.
  fac = pd.read_csv('전국지식산업센터_입주업체.csv')


,입지유형,회사명,단지명_x,전화번호,공장주소,주소(지번),생산품,대표업종,업종명,업종코드_전체,...,지목,용지면적,건축면적,제조면적,부대면적,공장대표주소(도로명),분양형태,건축상태,Unnamed: 17,Unnamed: 18
0,NaN,NaN,NaN,NaN,NaN,강원특별자치도 강릉시 사천면 방동리 805-3,NaN,NaN,NaN,NaN,...,주차장,7964.0,8586.96,4344.24,4242.72,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,강원특별자치도 동해시 구호동 216,NaN,NaN,NaN,NaN,...,공장용지,37830.7,0.00,0.00,0.00,강원특별자치도 동해시 공단12로 13 (구호동),NaN,미착공,NaN,NaN


In [26]:
kic.head(2)

,번호,센터명,주소,클러스터,위도,경도,은행,편의점,지하철 개수,지하철 점수,지하철등급,국도(km),고속도로(km),도로점수,도로등급
0,0,강릉 공공임대형 지식산업센터,강원특별자치도 강릉시 사천면 방동리 805-3,NaN,37.806207,128.854086,E,E,0,0.0,E,0.91,7.81,1.63,A
1,1,동해지식산업센터,강원특별자치도 동해시 구호동 216,NaN,37.481576,129.133811,C,E,0,0.0,E,3.65,6.42,4.65,E


In [27]:
print("입주업체 데이터 기본 정보:")
print(f"전체 입주업체 수: {len(fac)}")
print(f"지식산업센터 수: {fac['지식산업센터명'].nunique()}")
print(f"업종 종류 수: {fac['업종명'].nunique()}")

print("\n상위 10개 업종:")
print(fac['업종명'].value_counts().tail(30))

입주업체 데이터 기본 정보:
전체 입주업체 수: 51146
지식산업센터 수: 1520
업종 종류 수: 2033

상위 10개 업종:
업종명
그 외 기타 일반목적용 기계 제조업 외 6 종      1
치과용 기기 제조업 외 8 종               1
분사기 및 소화기 제조업 외 6 종            1
일반철물 제조업 외 3 종                 1
기타 목재가구 제조업 외 2 종              1
기체 여과기 제조업 외 6 종               1
내부 통신배선 공사업                    1
침구 및 관련제품 제조업 외 24 종           1
철물, 금속 파스너 및 수공구 도매업           1
가방 및 기타 보호용 케이스 제조업 외 27 종     1
침구 및 관련제품 제조업 외 3 종            1
증류기, 열교환기 및 가스발생기 제조업 외 5 종    1
기타 식품 첨가물 제조업 외 3 종            1
철강선 제조업 외 2 종                  1
귀금속 및 관련제품 제조업 외 3 종           1
액정 표시장치 제조업 외 2 종              1
기록매체 복제업 외 8 종                 1
산업용 트럭 및 적재기 제조업 외 3 종         1
방전램프용 안정기 제조업 외 8 종            1
날붙이 제조업 외 4 종                  1
일반 교과 학원                       1
배관 및 냉ㆍ난방 공사업 외 5 종            1
배관 및 냉ㆍ난방 공사업                  1
소방시설 공사업                       1
담배 소매업                         1
일반철물 제조업 외 6 종                 1
자동차 임대업 외 1 종                  1
그 외 기타 기계 및 장비 도매업             

업종이름 말고 업종코드로 클러스터링

In [30]:
fac['업종코드_리스트'] = fac['업종코드_전체'].fillna('').apply(lambda x: [code.strip() for code in str(x).split(',') if code.strip()])

center_code_analysis = fac.groupby('지식산업센터명').agg({
    '업종코드_리스트': lambda x: set(sum(x, [])),
    '회사명': 'count'
})

center_code_analysis['업종코드_다양성'] = center_code_analysis['업종코드_리스트'].apply(len)
center_code_analysis['업종코드_리스트'] = center_code_analysis['업종코드_리스트'].apply(list)
center_code_analysis = center_code_analysis.rename(columns={'회사명': '총_입주업체수'})
center_code_analysis = center_code_analysis.sort_values('총_입주업체수', ascending=False)

print("지식산업센터별 업종코드 다양성 상위 10개:")
print(center_code_analysis.head(10)[['총_입주업체수', '업종코드_다양성']])

지식산업센터별 업종코드 다양성 상위 10개:
                  총_입주업체수  업종코드_다양성
지식산업센터명                            
리드 스마트스퀘어 지식산업센터      915       231
국제신탁(주)SK테크노파크        718       164
센텀스카이비즈               683        94
가산어반워크1차              662        65
병점지식산업센터-2            596       253
(주)투게더홀딩스 지식산업센터      596       253
원스페이스                 596       253
(주)유앤미개발              596       253
병점지식산업센터-1            596       253
더챔버라티파니               596       253


In [31]:
industry_code_cooccurrence = defaultdict(int)

for center in fac['지식산업센터명'].unique():
    if pd.isna(center):
        continue
    
    # 해당 센터의 모든 업종코드 리스트 (중복 제거)
    codes = fac[fac['지식산업센터명'] == center]['업종코드_리스트'].sum()
    codes = list(set(codes))
    
    # 같은 센터 내 업종코드 조합 생성
    if len(codes) > 1:
        for combo in combinations(codes, 2):
            code_pair = tuple(sorted(combo))
            industry_code_cooccurrence[code_pair] += 1

# 가장 자주 함께 나타나는 업종코드 조합
print("가장 자주 함께 입주하는 업종코드 조합 TOP 10:")
sorted_code_cooccurrence = sorted(industry_code_cooccurrence.items(), key=lambda x: x[1], reverse=True)
for i, (pair, count) in enumerate(sorted_code_cooccurrence[:10]):
    print(f"{i+1}. {pair[0]} + {pair[1]}: {count}개 센터")

가장 자주 함께 입주하는 업종코드 조합 TOP 10:
1. 26421 + 26429: 307개 센터
2. 26410 + 26421: 296개 센터
3. 26410 + 26429: 295개 센터
4. 26429 + 28123: 275개 센터
5. 26421 + 28123: 266개 센터
6. 26421 + 26519: 258개 센터
7. 26410 + 28123: 248개 센터
8. 26429 + 26519: 247개 센터
9. 26299 + 28123: 243개 센터
10. 28123 + 28422: 243개 센터


In [32]:
import pandas as pd

# 엑셀 파일 읽기
df = pd.read_excel('한국표준산업분류.xlsx')

# 상위 5개 행 출력
df.head()

,"A 농업, 임업 및 어업 (01 ~ 03)\nAgriculture, forestry and fishing",Unnamed: 1,Unnamed: 2
0,NaN,NaN,NaN
1,01,농업,Agriculture
2,NaN,NaN,NaN
3,011,작물 재배업,Growing of Crops
4,NaN,NaN,NaN


---------------------

K - means(업종별)

In [ ]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import MultiLabelBinarizer
import folium
import matplotlib.pyplot as plt
import matplotlib
from geopy.geocoders import Nominatim
from tqdm import tqdm

# 1. 데이터 로드
fac = pd.read_csv('전국지식산업센터_입주업체.csv')
kic = pd.read_csv('kic.csv')  # 센터별 위도/경도/주소 포함
std_code = pd.read_excel('한국표준산업분류.xlsx', names=['코드', '항목명', '기타'], header=1)

# 2. 업종코드 리스트 생성
fac['업종코드_리스트'] = fac['업종코드_전체'].fillna('').apply(
    lambda x: [code.strip() for code in str(x).split(',') if code.strip()]
)

# 3. 센터별 업종코드 리스트 생성
center_codes_dict = {}
for center in fac['지식산업센터명'].unique():
    if pd.isna(center):
        continue
    codes = fac[fac['지식산업센터명'] == center]['업종코드_리스트'].sum()
    codes = list(set(codes))  # 중복 제거
    if codes:
        center_codes_dict[center] = codes

# 4. 원-핫 인코딩 및 클러스터링
mlb_code = MultiLabelBinarizer()
code_matrix = mlb_code.fit_transform(center_codes_dict.values())
center_names = list(center_codes_dict.keys())

n_clusters = 20
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
cluster_labels = kmeans.fit_predict(code_matrix)

# 5. 클러스터링 결과 저장
cluster_results = pd.DataFrame({
    '지식산업센터명': center_names,
    '클러스터': cluster_labels
})

# 6. 표준산업분류표 코드→항목명 매핑 딕셔너리 (NaN/공백 처리)
std_code = std_code[std_code['코드'].notna() & std_code['항목명'].notna()]
std_code['코드'] = std_code['코드'].astype(str).str.strip()
std_code['항목명'] = std_code['항목명'].astype(str).str.strip()
code2name = std_code.set_index('코드')['항목명'].to_dict()

# 7. 클러스터별 주요 업종코드 및 업종명 출력
print(f"\n업종코드 기반 클러스터링 결과 ({n_clusters}개 클러스터):")
for cluster_id in range(n_clusters):
    cluster_centers = cluster_results[cluster_results['클러스터'] == cluster_id]['지식산업센터명'].tolist()
    print(f"\n클러스터 {cluster_id} ({len(cluster_centers)}개 센터):")
    cluster_codes = []
    for center in cluster_centers:
        cluster_codes.extend(center_codes_dict[center])
    code_counts = pd.Series(cluster_codes).value_counts()
    top_codes = code_counts.head(5)
    top_code_names = {code: f"{code2name.get(str(code).strip(), 'N/A')} ({cnt}회)" for code, cnt in top_codes.items()}
    print(f"주요 업종코드+업종명: {top_code_names}")
    print(f"대표 센터: {cluster_centers[:3]}")

# 8. 클러스터 결과와 kic(위도경도) 데이터 병합
map_df = cluster_results.merge(
    kic[['센터명', '위도', '경도', '주소']], 
    left_on='지식산업센터명', right_on='센터명', how='left'
)

# 9. 위도/경도 없는 경우 주소로 지오코딩 (최소화)
geolocator = Nominatim(user_agent="kic_cluster_map")
lat_list, lon_list = [], []
for idx, row in tqdm(map_df.iterrows(), total=len(map_df)):
    lat, lon = row['위도'], row['경도']
    if pd.isna(lat) or pd.isna(lon):
        try:
            location = geolocator.geocode(row['주소'])
            if location:
                lat, lon = location.latitude, location.longitude
        except:
            lat, lon = None, None
    lat_list.append(lat)
    lon_list.append(lon)
map_df['위도'] = lat_list
map_df['경도'] = lon_list

# 10. 지도 시각화 (회색/흑백 배경, 클러스터별 FeatureGroup)
cmap = plt.get_cmap('tab20b', n_clusters)
colors = [matplotlib.colors.rgb2hex(cmap(i)) for i in range(n_clusters)]

m = folium.Map(location=[map_df['위도'].mean(), map_df['경도'].mean()], zoom_start=7, tiles='CartoDB positron')

feature_groups = []
for cluster_id in sorted(map_df['클러스터'].unique()):
    fg = folium.FeatureGroup(name=f"클러스터 {cluster_id}", show=True)
    color = colors[int(cluster_id) % len(colors)]
    for _, row in map_df[(map_df['클러스터'] == cluster_id) & map_df['위도'].notna() & map_df['경도'].notna()].iterrows():
        folium.CircleMarker(
            location=[row['위도'], row['경도']],
            radius=6,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.8,
            popup=f"{row['지식산업센터명']} (클러스터 {row['클러스터']})"
        ).add_to(fg)
    fg.add_to(m)
    feature_groups.append(fg)

# 11. 전체 클러스터 선택/해제용 FeatureGroup (cluster -1)
fg_all = folium.FeatureGroup(name="cluster -1 (전체 해제/선택)", show=False)
# 아무 마커도 추가하지 않음 (빈 그룹)
fg_all.add_to(m)

# 12. LayerControl (전체선택/전체해제 버튼)
folium.LayerControl(collapsed=False).add_to(m)

# 13. 안내문 팝업(전체선택/해제 방법)
folium.map.Marker(
    [map_df['위도'].max(), map_df['경도'].min()],
    icon=folium.DivIcon(html="""
        <div style="font-size: 14pt; color: black; background: white; border: 1px solid gray; padding: 4px;">
            <b>Tip:</b> 우측 상단 레이어박스에서<br>
            <b>클러스터별 선택/해제</b> 및<br>
            <b>cluster -1</b>을 클릭하면 전체 해제/선택이 가능합니다!
        </div>
    """)
).add_to(m)

m.save('K_cluster(업종별).html')
print("지도 저장 완료: K_cluster(업종별).html (회색 배경, 클러스터별 선택/전체해제 기능 포함)")

/var/folders/gh/rmj80qln5tz2v55hd102xqhr0000gn/T/ipykernel_65349/749477094.py:12: DtypeWarning: Columns (7,27) have mixed types. Specify dtype option on import or set low_memory=False.
  fac = pd.read_csv('전국지식산업센터_입주업체.csv')



업종코드 기반 클러스터링 결과 (20개 클러스터):

클러스터 0 (116개 센터):
주요 업종코드+업종명: {'26429': '기타 무선 통신장비 제조업 (107회)', '26421': '방송장비 제조업 (100회)', '26410': '유선 통신장비 제조업 (99회)', '28123': 'N/A (94회)', '26519': '비디오 및 기타 영상기기 제조업 (81회)'}
대표 센터: ['GL메트로시티 향동', '일산테크노타운', '디테크타워']

클러스터 1 (3개 센터):
주요 업종코드+업종명: {'27213': '물질 검사, 측정 및 분석기구 제조업 (3회)', '71531': '경영컨설팅업 (3회)', '73203': '시각 디자인업 (3회)', '26421': '방송장비 제조업 (3회)', '26429': '기타 무선 통신장비 제조업 (3회)'}
대표 센터: ['가산테라타워', '대륭테크노타운20차', '더스카이밸리']

클러스터 2 (6개 센터):
주요 업종코드+업종명: {'26429': '기타 무선 통신장비 제조업 (6회)', '27192': '정형외과용 및 신체보정용 기기 제조업 (6회)', '68112': '비주거용 건물 임대업 (6회)', '27212': '전자기 측정, 시험 및 분석기구 제조업 (6회)', '14191': '셔츠 및 체육복 제조업 (6회)'}
대표 센터: ['SK V1', 'SK엔테크노파크', '성남센터엠']

클러스터 3 (15개 센터):
주요 업종코드+업종명: {'26410': '유선 통신장비 제조업 (15회)', '28123': 'N/A (15회)', '26519': '비디오 및 기타 영상기기 제조업 (15회)', '26429': '기타 무선 통신장비 제조업 (15회)', '27215': '기기용 자동측정 및 제어장치 제조업 (15회)'}
대표 센터: ['삼송테크노밸리', '광명아파트형공장', '신일IT유토']

클러스터 4 (6개 센터):
주요 업종코드+업종명: {'29133': '탭, 밸브 및 유사장치 제조업 

100%|██████████| 1135/1135 [02:18<00:00,  8.18it/s]


지도 저장 완료: K_cluster(업종별).html (MarkerCluster, 회색 배경, 빠른 렌더링)


--------------------------------------

클러스터 K 분석 및 수정

------------------

지산 내부 업종코드들을 벡터화해서 유사도 클러스터링

In [4]:
import pandas as pd

# CSV 파일 읽기
df = pd.read_csv('전국지식산업센터_입주업체.csv')

df.head(2)

/var/folders/gh/rmj80qln5tz2v55hd102xqhr0000gn/T/ipykernel_65349/1557113611.py:4: DtypeWarning: Columns (7,27) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('전국지식산업센터_입주업체.csv')


,입지유형,회사명,단지명_x,전화번호,공장주소,주소(지번),생산품,대표업종,업종명,업종코드_전체,...,지목,용지면적,건축면적,제조면적,부대면적,공장대표주소(도로명),분양형태,건축상태,Unnamed: 17,Unnamed: 18
0,NaN,NaN,NaN,NaN,NaN,강원특별자치도 강릉시 사천면 방동리 805-3,NaN,NaN,NaN,NaN,...,주차장,7964.0,8586.96,4344.24,4242.72,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,강원특별자치도 동해시 구호동 216,NaN,NaN,NaN,NaN,...,공장용지,37830.7,0.00,0.00,0.00,강원특별자치도 동해시 공단12로 13 (구호동),NaN,미착공,NaN,NaN


In [6]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity

# 1. 데이터 로드
fac = pd.read_csv('전국지식산업센터_입주업체.csv')

# 2. 모든 업종코드 집합 만들기
all_codes = set()
fac['업종코드_리스트'] = fac['업종코드_전체'].fillna('').apply(
    lambda x: [code.strip() for code in str(x).split(',') if code.strip()]
)
for codes in fac['업종코드_리스트']:
    all_codes.update(codes)
all_codes = sorted(all_codes)
code2idx = {code: idx for idx, code in enumerate(all_codes)}
vector_dim = len(all_codes)

# 3. 지식산업센터별 업종코드 카운트 벡터 만들기
center_vectors = {}
for center in fac['지식산업센터명'].unique():
    if pd.isna(center):
        continue
    codes = fac[fac['지식산업센터명'] == center]['업종코드_리스트'].sum()
    vec = np.zeros(vector_dim, dtype=int)
    for code in codes:
        if code in code2idx:
            vec[code2idx[code]] += 1
    center_vectors[center] = vec

# 4. 벡터 행렬로 변환
center_names = list(center_vectors.keys())
X = np.stack([center_vectors[name] for name in center_names])

# 5. 클러스터링 (예: KMeans)
n_clusters = 40
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
cluster_labels = kmeans.fit_predict(X)

# 6. 결과 저장 및 확인
result = pd.DataFrame({
    '지식산업센터명': center_names,
    '클러스터': cluster_labels
})
print(result.head())

# (선택) 클러스터별 대표 업종코드 출력
for cluster_id in range(n_clusters):
    idxs = np.where(cluster_labels == cluster_id)[0]
    if len(idxs) == 0:
        continue
    cluster_vec = X[idxs].sum(axis=0)
    top_idx = cluster_vec.argsort()[::-1][:5]
    top_codes = [all_codes[i] for i in top_idx]
    top_counts = [cluster_vec[i] for i in top_idx]
    print(f"\n클러스터 {cluster_id} 대표 업종코드:")
    for code, cnt in zip(top_codes, top_counts):
        print(f"{code}: {cnt}회")

/var/folders/gh/rmj80qln5tz2v55hd102xqhr0000gn/T/ipykernel_65349/3919302020.py:7: DtypeWarning: Columns (7,27) have mixed types. Specify dtype option on import or set low_memory=False.
  fac = pd.read_csv('전국지식산업센터_입주업체.csv')


           지식산업센터명  클러스터
0  강릉 공공임대형 지식산업센터     0
1         동해지식산업센터     0
2           표준공장1동     0
3           표준공장2동     0
4           표준공장3동     0

클러스터 0 대표 업종코드:
28123: 289회
26429: 241회
26421: 236회
26410: 218회
28422: 205회

클러스터 1 대표 업종코드:
72121: 456회
68122: 449회
58222: 33회
71310: 25회
71531: 22회

클러스터 2 대표 업종코드:
68112: 253회
71531: 105회
72129: 45회
71310: 42회
58222: 32회

클러스터 3 대표 업종코드:
58222: 223회
71531: 164회
73202: 123회
71310: 107회
62010: 89회

클러스터 4 대표 업종코드:
58221: 249회
58222: 216회
71531: 14회
73203: 6회
72111: 4회

클러스터 5 대표 업종코드:
68112: 333회
58222: 57회
71531: 54회
58221: 32회
72129: 31회

클러스터 6 대표 업종코드:
71531: 326회
68112: 233회
72111: 42회
71310: 41회
58222: 40회

클러스터 7 대표 업종코드:
25924: 162회
28123: 139회
27216: 47회
25929: 45회
29294: 44회

클러스터 8 대표 업종코드:
29294: 252회
25924: 238회
25929: 175회
29271: 126회
68112: 126회

클러스터 9 대표 업종코드:
62090: 165회
58222: 85회
58221: 82회
72129: 67회
62010: 58회

클러스터 10 대표 업종코드:
58222: 280회
71531: 40회
58221: 22회
68112: 18회
62010: 6회

클러스터 11 대표 업종코드:
58222: 617회
715

In [10]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import folium
import matplotlib.pyplot as plt
import matplotlib
from geopy.geocoders import Nominatim
from tqdm import tqdm
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN

fac = pd.read_csv('전국지식산업센터_입주업체.csv')
kic = pd.read_csv('kic.csv')
std_code = pd.read_excel('한국표준산업분류.xlsx', names=['코드', '항목명', '기타'], header=1)

# 2. 업종코드 리스트 생성
fac['업종코드_리스트'] = fac['업종코드_전체'].fillna('').apply(
    lambda x: [code.strip() for code in str(x).split(',') if code.strip()]
)

# 3. 전체 업종코드 집합 및 인덱스 매핑
all_codes = set()
for codes in fac['업종코드_리스트']:
    all_codes.update(codes)
all_codes = sorted(all_codes)
code2idx = {code: idx for idx, code in enumerate(all_codes)}
vector_dim = len(all_codes)

# 4. 지식산업센터별 업종코드 카운트 벡터 만들기
center_vectors = {}
for center in fac['지식산업센터명'].unique():
    if pd.isna(center):
        continue
    codes = fac[fac['지식산업센터명'] == center]['업종코드_리스트'].sum()
    vec = np.zeros(vector_dim, dtype=int)
    for code in codes:
        if code in code2idx:
            vec[code2idx[code]] += 1
    center_vectors[center] = vec

# 5. 벡터 행렬로 변환
center_names = list(center_vectors.keys())
X = np.stack([center_vectors[name] for name in center_names])

#----------------------------------------------------------------------------------------------------
dbscan = DBSCAN(eps=3, min_samples=2)  # 파라미터는 데이터에 따라 조정
cluster_labels = dbscan.fit_predict(X)
# 7. 결과 저장
result = pd.DataFrame({
    '지식산업센터명': center_names,
    '클러스터': cluster_labels
})

# 8. 업종코드→업종명 매핑 딕셔너리
std_code = std_code[std_code['코드'].notna() & std_code['항목명'].notna()]
std_code['코드'] = std_code['코드'].astype(str).str.strip()
std_code['항목명'] = std_code['항목명'].astype(str).str.strip()
code2name = std_code.set_index('코드')['항목명'].to_dict()

# 9. 클러스터별 대표 업종코드+업종명 출력
print(f"\n업종코드 기반 클러스터링 결과 ({n_clusters}개 클러스터):")
for cluster_id in range(n_clusters):
    idxs = np.where(cluster_labels == cluster_id)[0]
    if len(idxs) == 0:
        continue
    cluster_vec = X[idxs].sum(axis=0)
    top_idx = cluster_vec.argsort()[::-1][:5]
    top_codes = [all_codes[i] for i in top_idx]
    top_counts = [cluster_vec[i] for i in top_idx]
    top_code_names = {code: f"{code2name.get(str(code).strip(), 'N/A')} ({cnt}회)" for code, cnt in zip(top_codes, top_counts)}
    print(f"\n클러스터 {cluster_id} 대표 업종코드+업종명: {top_code_names}")
    print(f"대표 센터: {[center_names[i] for i in idxs[:3]]}")

# 10. 클러스터 결과와 kic(위도경도) 데이터 병합
map_df = result.merge(
    kic[['센터명', '위도', '경도', '주소']],
    left_on='지식산업센터명', right_on='센터명', how='left'
)

# 11. 위도/경도 없는 경우 주소로 지오코딩 (최소화)
geolocator = Nominatim(user_agent="kic_cluster_map")
lat_list, lon_list = [], []
for idx, row in tqdm(map_df.iterrows(), total=len(map_df)):
    lat, lon = row['위도'], row['경도']
    if pd.isna(lat) or pd.isna(lon):
        try:
            location = geolocator.geocode(row['주소'])
            if location:
                lat, lon = location.latitude, location.longitude
        except:
            lat, lon = None, None
    lat_list.append(lat)
    lon_list.append(lon)
map_df['위도'] = lat_list
map_df['경도'] = lon_list

# 12. 지도 시각화 (회색/흑백 배경, 클러스터별 FeatureGroup)
cmap = plt.get_cmap('tab20b', n_clusters)
colors = [matplotlib.colors.rgb2hex(cmap(i)) for i in range(n_clusters)]

m = folium.Map(location=[map_df['위도'].mean(), map_df['경도'].mean()], zoom_start=7, tiles='CartoDB positron')

feature_groups = []
for cluster_id in sorted(map_df['클러스터'].unique()):
    fg = folium.FeatureGroup(name=f"클러스터 {cluster_id}", show=False)  # show=False로 기본 해제
    color = colors[int(cluster_id) % len(colors)]
    for _, row in map_df[(map_df['클러스터'] == cluster_id) & map_df['위도'].notna() & map_df['경도'].notna()].iterrows():
        folium.CircleMarker(
            location=[row['위도'], row['경도']],
            radius=6,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.8,
            popup=f"{row['지식산업센터명']} (클러스터 {row['클러스터']})"
        ).add_to(fg)
    fg.add_to(m)
    feature_groups.append(fg)

# 13. 전체 클러스터 선택/해제용 FeatureGroup (cluster -1)
fg_all = folium.FeatureGroup(name="cluster -1 (전체 해제/선택)", show=True)
fg_all.add_to(m)

# 14. LayerControl (전체선택/전체해제 버튼)
folium.LayerControl(collapsed=False).add_to(m)

# 15. 안내문 팝업(전체선택/해제 방법)
folium.map.Marker(
    [map_df['위도'].max(), map_df['경도'].min()],
    icon=folium.DivIcon(html="""
        <div style="font-size: 14pt; color: black; background: white; border: 1px solid gray; padding: 4px;">
            <b>Tip:</b> 우측 상단 레이어박스에서<br>
            <b>클러스터별 선택/해제</b> 및<br>
            <b>cluster -1</b>을 클릭하면 전체 해제/선택이 가능합니다!<br>
            (기본적으로 모든 클러스터는 해제된 상태로 시작합니다)
        </div>
    """)
).add_to(m)

m.save('K_cluster(업종별)_countvector.html')
print("지도 저장 완료: K_cluster(업종별)_countvector.html (회색 배경, 클러스터별 기본 해제/전체해제 기능 포함)")

/var/folders/gh/rmj80qln5tz2v55hd102xqhr0000gn/T/ipykernel_65349/1409242045.py:12: DtypeWarning: Columns (7,27) have mixed types. Specify dtype option on import or set low_memory=False.
  fac = pd.read_csv('전국지식산업센터_입주업체.csv')



업종코드 기반 클러스터링 결과 (20개 클러스터):

클러스터 0 대표 업종코드+업종명: {'26421': '방송장비 제조업 (64회)', '26429': '기타 무선 통신장비 제조업 (57회)', '26410': '유선 통신장비 제조업 (56회)', '68112': '비주거용 건물 임대업 (50회)', '28123': 'N/A (50회)'}
대표 센터: ['강릉 공공임대형 지식산업센터', '동해지식산업센터', '양양군 현남면 북분리 425-7, 460-2']

클러스터 1 대표 업종코드+업종명: {'17902': '위생용 종이제품 제조업 (9회)', '10792': '차류 가공업 (6회)', '28123': 'N/A (6회)', '20423': '인쇄잉크 제조업 (6회)', '34020': 'N/A (6회)'}
대표 센터: ['표준공장1동', '표준공장2동', '표준공장3동']

클러스터 2 대표 업종코드+업종명: {'14112': '여자용 정장 제조업 (22회)', '14120': '내의 및 잠옷 제조업 (22회)', '14192': '근무복, 작업복 및 유사의복 제조업 (22회)', '14199': '그외 기타 봉제의복 제조업 (22회)', '14111': '남자용 정장 제조업 (20회)'}
대표 센터: ['동두천시 싸이언스타워', '동두천싸이언스타워 두드림 패션센터']

클러스터 3 대표 업종코드+업종명: {'58212': 'N/A (66회)', '58222': '응용소프트웨어 개발 및 공급업 (44회)', '58219': '기타 게임 소프트웨어 개발 및 공급업 (22회)', '58221': '시스템  소프트웨어 개발 및 공급업 (14회)', '70113': '의학 및 약학 연구개발업 (10회)'}
대표 센터: ['LH 기업성장센터', 'LH제2기업성장센터']

클러스터 4 대표 업종코드+업종명: {'29294': '주형 및 금형 제조업 (252회)', '25924': '절삭가공 및 유사처리업 (238회)', '25929': '그외 기타 금속가공업 (

100%|██████████| 1605/1605 [04:05<00:00,  6.54it/s]


지도 저장 완료: K_cluster(업종별)_countvector.html (회색 배경, 클러스터별 기본 해제/전체해제 기능 포함)
